In [1]:
pip install nvcc4jupyter


In [2]:
%load_ext nvcc4jupyter


Source files will be saved in "/tmp/tmpvi7dgsex".


In [6]:
%%cuda
#include <iostream>
int main(){
    std::cout << "hello" << std::endl;
    return 0;
}

hello



test code  for  creating two square matrices with dimensions of 1000 x 1000 elements. All elements of which are 1.

In [31]:
%%cuda

#include <stdio.h>

#define N 1000

__global__ void initializeMatrix(int *matrix) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        int index = row * N + col;
        matrix[index] = 1;
    }
}

int main() {
    int *h_matrix1, *h_matrix2; // Host matrices
    int *d_matrix1, *d_matrix2; // Device matrices

    // Allocate memory for host matrices
    h_matrix1 = (int*)malloc(N * N * sizeof(int));
    h_matrix2 = (int*)malloc(N * N * sizeof(int));

    // Allocate memory for device matrices
    cudaMalloc((void**)&d_matrix1, N * N * sizeof(int));
    cudaMalloc((void**)&d_matrix2, N * N * sizeof(int));

    // Define grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);

    // Initialize matrices on device
    initializeMatrix<<<gridDim, blockDim>>>(d_matrix1);
    initializeMatrix<<<gridDim, blockDim>>>(d_matrix2);

    // Copy matrices from device to host
    cudaMemcpy(h_matrix1, d_matrix1, N * N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_matrix2, d_matrix2, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Display the matrices (optional)
    printf("Matrix 1:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", h_matrix1[i * N + j]);
        }
        printf("\n");
    }

    printf("\nMatrix 2:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", h_matrix2[i * N + j]);
        }
        printf("\n");
    }

    // Free memory
    free(h_matrix1);
    free(h_matrix2);
    cudaFree(d_matrix1);
    cudaFree(d_matrix2);

    return 0;
}



Matrix 1:
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



int 32

In [40]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <curand.h>
#include <curand_kernel.h>
#include <sys/time.h>

#define N 10000
#define BLOCK_SIZE 16

__global__ void matrixMultiplication(int *a, int *b, int *c, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < n && col < n) {
        int sum = 0;
        for (int i = 0; i < n; i++) {
            sum += a[row * n + i] * b[i * n + col];
        }
        c[row * n + col] = sum;
    }
}

void fillMatrix(int *matrix, int n) {
    curandGenerator_t gen;
    curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);
    curandGenerate(gen, (unsigned int*)matrix, n * n);
    curandDestroyGenerator(gen);
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int matrixSize = N * N * sizeof(int);

    // Allocate host memory
    a = (int*)malloc(matrixSize);
    b = (int*)malloc(matrixSize);
    c = (int*)malloc(matrixSize);

    // Allocate device memory
    cudaMalloc(&d_a, matrixSize);
    cudaMalloc(&d_b, matrixSize);
    cudaMalloc(&d_c, matrixSize);

    // Step 1: Start timer for memory allocation and matrix creation
    struct timeval start, end;
    gettimeofday(&start, NULL);

    // Step 2: Fill matrices with random values
    fillMatrix(a, N);
    fillMatrix(b, N);

    // Step 2: Stop timer for memory allocation and matrix creation
    gettimeofday(&end, NULL);
    double elapsedTime = (end.tv_sec - start.tv_sec) * 1000.0; // sec to ms
    elapsedTime += (end.tv_usec - start.tv_usec) / 1000.0;      // us to ms
    printf("Matrix creation and filling time: %.3f ms\n", elapsedTime);

    // Step 3: Start timer for matrix multiplication
    gettimeofday(&start, NULL);

    // Copy matrices from host to device
    cudaMemcpy(d_a, a, matrixSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, matrixSize, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 dimGrid((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

    // Launch kernel
    matrixMultiplication<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);

    // Copy result back to host
    cudaMemcpy(c, d_c, matrixSize, cudaMemcpyDeviceToHost);

    // Step 3: Stop timer for matrix multiplication
    gettimeofday(&end, NULL);
    elapsedTime = (end.tv_sec - start.tv_sec) * 1000.0; // sec to ms
    elapsedTime += (end.tv_usec - start.tv_usec) / 1000.0;      // us to ms
    printf("Matrix multiplication time: %.3f ms\n", elapsedTime);

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(a);
    free(b);
    free(c);

    return 0;
}



/usr/bin/ld: /tmp/tmpxft_00002ce6_00000000-11_single_file.o: in function `fillMatrix(int*, int)':
tmpxft_00002ce6_00000000-6_single_file.cudafe1.cpp:(.text+0x49): undefined reference to `curandCreateGenerator'
/usr/bin/ld: tmpxft_00002ce6_00000000-6_single_file.cudafe1.cpp:(.text+0x5a): undefined reference to `curandSetPseudoRandomGeneratorSeed'
/usr/bin/ld: tmpxft_00002ce6_00000000-6_single_file.cudafe1.cpp:(.text+0x76): undefined reference to `curandGenerate'
/usr/bin/ld: tmpxft_00002ce6_00000000-6_single_file.cudafe1.cpp:(.text+0x82): undefined reference to `curandDestroyGenerator'
collect2: error: ld returned 1 exit status



int 32 10000 x 10000 error

so convert 10000 to 1000

In [47]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>

#define N 1000
#define THREADS_PER_BLOCK 16

// Kernel function to perform matrix multiplication
__global__ void matrixMultiply(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int sum = 0;
    for (int k = 0; k < N; ++k) {
        sum += a[row * N + k] * b[k * N + col];
    }
    c[row * N + col] = sum;
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * N * sizeof(int);

    // Allocate memory for host matrices
    a = (int *)malloc(size);
    b = (int *)malloc(size);
    c = (int *)malloc(size);

    // Allocate memory for device matrices
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start[3], end[3];
    start[0] = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    end[0] = clock();

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start[1] = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = rand() % 10;
        b[i] = rand() % 10;
    }
    end[1] = clock();

    // Step 3: Matrix multiplication
    printf("Step 3: Performing matrix multiplication...\n");
    start[2] = clock();
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(THREADS_PER_BLOCK, THREADS_PER_BLOCK);
    dim3 numBlocks(N / THREADS_PER_BLOCK, N / THREADS_PER_BLOCK);
    matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_a, d_b, d_c);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    end[2] = clock();

    // Calculate elapsed time for each part
    double duration[3];
    for (int i = 0; i < 3; ++i) {
        duration[i] = ((double)(end[i] - start[i])) / CLOCKS_PER_SEC;
        printf("Execution time for step %d: %f seconds\n", i + 1, duration[i]);
    }

    // Calculate sum of elapsed times
    double total_duration = 0;
    for (int i = 0; i < 3; ++i) {
        total_duration += duration[i];
    }
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}



Step 1: Creating matrices...
Step 2: Filling matrices with random values...
Step 3: Performing matrix multiplication...
Execution time for step 1: 0.000002 seconds
Execution time for step 2: 0.048678 seconds
Execution time for step 3: 0.000022 seconds
Total execution time: 0.048702 seconds



float 32 1000*1000

In [48]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>

#define N 1000
#define THREADS_PER_BLOCK 16

// Kernel function to perform matrix multiplication
__global__ void matrixMultiply(float *a, float *b, float *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float sum = 0;
    for (int k = 0; k < N; ++k) {
        sum += a[row * N + k] * b[k * N + col];
    }
    c[row * N + col] = sum;
}

int main() {
    float *a, *b, *c;
    float *d_a, *d_b, *d_c;
    int size = N * N * sizeof(float);

    // Allocate memory for host matrices
    a = (float *)malloc(size);
    b = (float *)malloc(size);
    c = (float *)malloc(size);

    // Allocate memory for device matrices
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start[3], end[3];
    start[0] = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    end[0] = clock();

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start[1] = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = (float)rand() / RAND_MAX;  // Random float value between 0 and 1
        b[i] = (float)rand() / RAND_MAX;  // Random float value between 0 and 1
    }
    end[1] = clock();

    // Step 3: Matrix multiplication
    printf("Step 3: Performing matrix multiplication...\n");
    start[2] = clock();
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(THREADS_PER_BLOCK, THREADS_PER_BLOCK);
    dim3 numBlocks(N / THREADS_PER_BLOCK, N / THREADS_PER_BLOCK);
    matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_a, d_b, d_c);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    end[2] = clock();

    // Calculate elapsed time for each part
    double duration[3];
    for (int i = 0; i < 3; ++i) {
        duration[i] = ((double)(end[i] - start[i])) / CLOCKS_PER_SEC;
        printf("Execution time for step %d: %f seconds\n", i + 1, duration[i]);
    }

    // Calculate sum of elapsed times
    double total_duration = 0;
    for (int i = 0; i < 3; ++i) {
        total_duration += duration[i];
    }
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


Step 1: Creating matrices...
Step 2: Filling matrices with random values...
Step 3: Performing matrix multiplication...
Execution time for step 1: 0.000001 seconds
Execution time for step 2: 0.045096 seconds
Execution time for step 3: 0.000016 seconds
Total execution time: 0.045113 seconds



float 64 1000*1000

In [50]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>

#define N 1000
#define THREADS_PER_BLOCK 16

// Kernel function to perform matrix multiplication
__global__ void matrixMultiply(double *a, double *b, double *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    double sum = 0;
    for (int k = 0; k < N; ++k) {
        sum += a[row * N + k] * b[k * N + col];
    }
    c[row * N + col] = sum;
}

int main() {
    double *a, *b, *c;
    double *d_a, *d_b, *d_c;
    int size = N * N * sizeof(double);

    // Allocate memory for host matrices
    a = (double *)malloc(size);
    b = (double *)malloc(size);
    c = (double *)malloc(size);

    // Allocate memory for device matrices
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start[3], end[3];
    start[0] = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    end[0] = clock();

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start[1] = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = (double)rand() / RAND_MAX;  // Random double value between 0 and 1
        b[i] = (double)rand() / RAND_MAX;  // Random double value between 0 and 1
    }
    end[1] = clock();

    // Step 3: Matrix multiplication
    printf("Step 3: Performing matrix multiplication...\n");
    start[2] = clock();
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(THREADS_PER_BLOCK, THREADS_PER_BLOCK);
    dim3 numBlocks(N / THREADS_PER_BLOCK, N / THREADS_PER_BLOCK);
    matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_a, d_b, d_c);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    end[2] = clock();

    // Calculate elapsed time for each part
    double duration[3];
    for (int i = 0; i < 3; ++i) {
        duration[i] = ((double)(end[i] - start[i])) / CLOCKS_PER_SEC;
        printf("Execution time for step %d: %f seconds\n", i + 1, duration[i]);
    }

    // Calculate sum of elapsed times
    double total_duration = 0;
    for (int i = 0; i < 3; ++i) {
        total_duration += duration[i];
    }
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


Step 1: Creating matrices...
Step 2: Filling matrices with random values...
Step 3: Performing matrix multiplication...
Execution time for step 1: 0.000001 seconds
Execution time for step 2: 0.049918 seconds
Execution time for step 3: 0.000011 seconds
Total execution time: 0.049930 seconds



step 4


In [52]:

%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define N 1000
#define THREADS_PER_BLOCK 16

// Kernel function to perform matrix multiplication
__global__ void matrixMultiply(float *a, float *b, float *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float sin_val = sinf(a[row * N + col]);
    float cos_val = cosf(b[row * N + col]);
    c[row * N + col] = sin_val * cos_val;
}

int main() {
    float *a, *b, *c;
    float *d_a, *d_b, *d_c;
    int size = N * N * sizeof(float);

    // Allocate memory for host matrices
    a = (float *)malloc(size);
    b = (float *)malloc(size);
    c = (float *)malloc(size);

    // Allocate memory for device matrices
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start[3], end[3];
    start[0] = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    end[0] = clock();

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start[1] = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = (float)rand() / RAND_MAX * 2.0f - 1.0f;  // Random float value between -1 and 1
        b[i] = (float)rand() / RAND_MAX * 2.0f - 1.0f;  // Random float value between -1 and 1
    }
    end[1] = clock();

    // Step 3: Matrix multiplication
    printf("Step 3: Performing matrix multiplication...\n");
    start[2] = clock();
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(THREADS_PER_BLOCK, THREADS_PER_BLOCK);
    dim3 numBlocks(N / THREADS_PER_BLOCK, N / THREADS_PER_BLOCK);
    matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_a, d_b, d_c);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    end[2] = clock();

    // Calculate elapsed time for each part
    double duration[3];
    for (int i = 0; i < 3; ++i) {
        duration[i] = ((double)(end[i] - start[i])) / CLOCKS_PER_SEC;
        printf("Execution time for step %d: %f seconds\n", i + 1, duration[i]);
    }

    // Calculate sum of elapsed times
    double total_duration = 0;
    for (int i = 0; i < 3; ++i) {
        total_duration += duration[i];
    }
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


Step 1: Creating matrices...
Step 2: Filling matrices with random values...
Step 3: Performing matrix multiplication...
Execution time for step 1: 0.000001 seconds
Execution time for step 2: 0.045965 seconds
Execution time for step 3: 0.000015 seconds
Total execution time: 0.045981 seconds



no GPU just CPU


 int32 1000*1000

In [53]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define N 1000

// Function to perform matrix multiplication on CPU
void matrixMultiply(int32_t *a, int32_t *b, int32_t *c) {
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            int32_t sum = 0;
            for (int k = 0; k < N; ++k) {
                sum += a[i * N + k] * b[k * N + j];
            }
            c[i * N + j] = sum;
        }
    }
}

int main() {
    int32_t *a, *b, *c;
    int32_t *d_a, *d_b, *d_c;
    int size = N * N * sizeof(int32_t);

    // Allocate memory for host matrices
    a = (int32_t *)malloc(size);
    b = (int32_t *)malloc(size);
    c = (int32_t *)malloc(size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    clock_t end = clock();
    double duration_step1 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 1: %f seconds\n", duration_step1);

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = rand() % 10;
        b[i] = rand() % 10;
    }
    end = clock();
    double duration_step2 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 2: %f seconds\n", duration_step2);

    // Step 3: Matrix multiplication
    printf("Step 3: Performing matrix multiplication...\n");
    start = clock();
    matrixMultiply(a, b, c);
    end = clock();
    double duration_step3 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 3: %f seconds\n", duration_step3);

    // Calculate total execution time
    double total_duration = duration_step1 + duration_step2 + duration_step3;
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);

    return 0;
}


Step 1: Creating matrices...
Execution time for step 1: 0.000000 seconds
Step 2: Filling matrices with random values...
Execution time for step 2: 0.049213 seconds
Step 3: Performing matrix multiplication...
Execution time for step 3: 6.394480 seconds
Total execution time: 6.443693 seconds



 float32 1000*1000

In [54]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define N 1000

// Function to perform matrix multiplication on CPU
void matrixMultiply(float *a, float *b, float *c) {
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            float sum = 0;
            for (int k = 0; k < N; ++k) {
                sum += a[i * N + k] * b[k * N + j];
            }
            c[i * N + j] = sum;
        }
    }
}

int main() {
    float *a, *b, *c;
    int size = N * N * sizeof(float);

    // Allocate memory for host matrices
    a = (float *)malloc(size);
    b = (float *)malloc(size);
    c = (float *)malloc(size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    clock_t end = clock();
    double duration_step1 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 1: %f seconds\n", duration_step1);

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = (float)(rand() % 10);
        b[i] = (float)(rand() % 10);
    }
    end = clock();
    double duration_step2 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 2: %f seconds\n", duration_step2);

    // Step 3: Matrix multiplication
    printf("Step 3: Performing matrix multiplication...\n");
    start = clock();
    matrixMultiply(a, b, c);
    end = clock();
    double duration_step3 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 3: %f seconds\n", duration_step3);

    // Calculate total execution time
    double total_duration = duration_step1 + duration_step2 + duration_step3;
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);

    return 0;
}


Step 1: Creating matrices...
Execution time for step 1: 0.000001 seconds
Step 2: Filling matrices with random values...
Execution time for step 2: 0.047826 seconds
Step 3: Performing matrix multiplication...
Execution time for step 3: 6.285575 seconds
Total execution time: 6.333402 seconds



 float32 1000*1000

In [55]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define N 1000

// Function to perform matrix multiplication on CPU
void matrixMultiply(double *a, double *b, double *c) {
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            double sum = 0;
            for (int k = 0; k < N; ++k) {
                sum += a[i * N + k] * b[k * N + j];
            }
            c[i * N + j] = sum;
        }
    }
}

int main() {
    double *a, *b, *c;
    int size = N * N * sizeof(double);

    // Allocate memory for host matrices
    a = (double *)malloc(size);
    b = (double *)malloc(size);
    c = (double *)malloc(size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    clock_t end = clock();
    double duration_step1 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 1: %f seconds\n", duration_step1);

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = (double)(rand() % 10);
        b[i] = (double)(rand() % 10);
    }
    end = clock();
    double duration_step2 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 2: %f seconds\n", duration_step2);

    // Step 3: Matrix multiplication
    printf("Step 3: Performing matrix multiplication...\n");
    start = clock();
    matrixMultiply(a, b, c);
    end = clock();
    double duration_step3 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 3: %f seconds\n", duration_step3);

    // Calculate total execution time
    double total_duration = duration_step1 + duration_step2 + duration_step3;
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);

    return 0;
}


Step 1: Creating matrices...
Execution time for step 1: 0.000000 seconds
Step 2: Filling matrices with random values...
Execution time for step 2: 0.053337 seconds
Step 3: Performing matrix multiplication...
Execution time for step 3: 9.274695 seconds
Total execution time: 9.328032 seconds



step 4

In [58]:
%%cuda


#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define N 1000

// Function to perform sin/cos operations on CPU
void matrixSinCos(float *a, float *b, float *c) {
    for (int i = 0; i < N * N; ++i) {
        c[i] = sinf(a[i]) * cosf(b[i]);
    }
}

int main() {
    float *a, *b, *c;
    int size = N * N * sizeof(float);

    // Allocate memory for host matrices
    a = (float *)malloc(size);
    b = (float *)malloc(size);
    c = (float *)malloc(size);

    // Step 1: Creation of matrices
    printf("Step 1: Creating matrices...\n");
    clock_t start = clock();
    // No need to explicitly create matrices, as they are initialized randomly
    clock_t end = clock();
    double duration_step1 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 1: %f seconds\n", duration_step1);

    // Step 2: Fill matrices with random values
    printf("Step 2: Filling matrices with random values...\n");
    start = clock();
    for (int i = 0; i < N * N; ++i) {
        a[i] = (float)(rand() % 10);
        b[i] = (float)(rand() % 10);
    }
    end = clock();
    double duration_step2 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 2: %f seconds\n", duration_step2);

    // Step 3: Sin/Cos operations
    printf("Step 3: Performing sin/cos operations...\n");
    start = clock();
    matrixSinCos(a, b, c);
    end = clock();
    double duration_step3 = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Execution time for step 3: %f seconds\n", duration_step3);

    // Calculate total execution time
    double total_duration = duration_step1 + duration_step2 + duration_step3;
    printf("Total execution time: %f seconds\n", total_duration);

    // Free memory
    free(a);
    free(b);
    free(c);

    return 0;
}



Step 1: Creating matrices...
Execution time for step 1: 0.000001 seconds
Step 2: Filling matrices with random values...
Execution time for step 2: 0.047700 seconds
Step 3: Performing sin/cos operations...
Execution time for step 3: 0.031550 seconds
Total execution time: 0.079251 seconds

